# Reglas de asociación

Las reglas de asociación son un método de minería de datos que se utiliza para encontrar relaciones interesantes entre variables en grandes bases de datos. 
Estas regrlas tienen la forma *"si A entonces B"*, lo que significa que si ocurre un determinado conjunto de elementos (A), es probable que también ocurra otro conjunto de elementos (B).

Las reglas de asociación son una de las técnicas más utilizadas en la industria minorista, y cuentan con una amplia variedad de aplicaciones, entre las que destacan:
 - El **análisis de la cesta de la compra**: Identificar qué productos se compran con frecuencia juntos en los supermercados. Esto puede ayudar a colocar los productos de forma estratégica para aumentar las ventas. Por ejemplo, una regla de asociación podría ser "si se compra pan, entonces también se compra leche". Uno de los mayores ejemplos del análisis de reglas de asociación es la correlación entre la cerveza y los pañales. Cuando Walmart, una cadena de tiendas en Estados Unidos, estudió el comportamiento de compra de sus clientes, se encontró con que los pañales y las cervezas se compran juntos. La explicación resultó ser que muchos los padres son los encargados de hacer las compras mientras las madres se quedan con el bebé.
 - Algunos **sistemas de recomendación**: Recomendar productos o contenido a los usuarios en función de sus comportamientos o preferencias pasadas. Por ejemplo, en un servicio de streaming de vídeo, si un usuario ha visto muchas películas de ciencia ficción, el sistema puede recomendar otras películas del mismo género. Algoritmos basados en reglas de asociación se usan en empresas como Spotify, Netflix y YouTube.
 - **Diagnóstico médico**: Se pueden usar reglas de asociación para identificar relaciones entre síntomas y enfermedades. Esto puede ayudar a los médicos a realizar diagnósticos más precisos.  

In [1]:
%pip install mlxtend pandas

Note: you may need to restart the kernel to use updated packages.


## Algoritmo Apriori
El algoritmo más popular y clásico, donde los datos se evalúan con reglas de asociación booleanas, es el denominado *Algoritmo a priori*. 
Este algoritmo determina grupos de productos que aparecen frecuentemente, y luego busca relaciones fuertes entre estos y otros productos.

In [2]:
import pandas as pd

df = pd.read_csv('./Datos/datos.csv', names = ['productos'], sep = ',')
df.head()

,productos
0,"MILK,BREAD,BISCUIT"
1,"BREAD,MILK,BISCUIT,CORNFLAKES"
2,"BREAD,TEA,BOURNVITA"
3,"JAM,MAGGI,BREAD,MILK"
4,"MAGGI,TEA,BISCUIT"


Lo primero que debemos hacer es separar cada fila del fichero por las comas, de forma que podamos acceder a los productos individuales comprados por cada cliente.

In [3]:
datos = list(df["productos"].apply(lambda x:x.split(",")))
datos

[['MILK', 'BREAD', 'BISCUIT'],
 ['BREAD', 'MILK', 'BISCUIT', 'CORNFLAKES'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['JAM', 'MAGGI', 'BREAD', 'MILK'],
 ['MAGGI', 'TEA', 'BISCUIT'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['MAGGI', 'TEA', 'CORNFLAKES'],
 ['MAGGI', 'BREAD', 'TEA', 'BISCUIT'],
 ['JAM', 'MAGGI', 'BREAD', 'TEA'],
 ['BREAD', 'MILK'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'SUGER', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'COCK'],
 ['BREAD', 'SUGER', 'BISCUIT'],
 ['COFFEE', 'SUGER', 'CORNFLAKES'],
 ['BREAD', 'SUGER', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'SUGER'],
 ['BREAD', 'COFFEE', 'SUGER'],
 ['TEA', 'MILK', 'COFFEE', 'CORNFLAKES']]

Lo siguiente que tenemos que hacer es generar un dataframe con el mismo numero de columnas para cada compra, y con valores verdadero/falso para cada una de esas columnas. Es decir, necesitamos realizar una codificación OneHot.
 
Para ello usaremos un `TransactionEncoder`. Los productos que los clientes compran en una transacción se representarán con el 1, mientras que los que no se compran aparecerán con un 0. 

In [4]:
from mlxtend.preprocessing import TransactionEncoder

encoder = TransactionEncoder()
datos_codificados = encoder.fit_transform(datos)
df = pd.DataFrame(datos_codificados, columns=encoder.columns_)

df

,BISCUIT,BOURNVITA,BREAD,COCK,COFFEE,CORNFLAKES,JAM,MAGGI,MILK,SUGER,TEA
0,True,False,True,False,False,False,False,False,True,False,False
1,True,False,True,False,False,True,False,False,True,False,False
2,False,True,True,False,False,False,False,False,False,False,True
3,False,False,True,False,False,False,True,True,True,False,False
4,True,False,False,False,False,False,False,True,False,False,True
5,False,True,True,False,False,False,False,False,False,False,True
6,False,False,False,False,False,True,False,True,False,False,True
7,True,False,True,False,False,False,False,True,False,False,True
8,False,False,True,False,False,False,True,True,False,False,True
9,False,False,True,False,False,False,False,False,True,False,False


Tras estas transformaciones, podemos ver facilmente que, por ejemplo, 7 clientes han comprado galletas, 8 han comprado café, y 5 han comprado leche.

### Generación de itemsets fecuentes con el modelo Apriori

El siguiente paso para la extracción de las reglas de asociación es obtener los itemsets frecuentes. En este caso emplearemos un modelo [Apriori](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/). Este algoritmo basa su funcionamiento en la generación iterativa de candidatos y su podado segun la frecuencia de ese candidato.

Cuando creamos un modelo Apriori debemos especificar el soporte mínimo, es decir, la frecuencia relativa de aparición de un conjunto de elementos en el conjunto de datos de transacciones a partir de la cual consideraremos el itemset frecuente:

$$
Soporte(I)= \frac{\text{Numero de transacciones en las que aparece el itemset }I}{\text{Numero de transacciones total}} 
$$
​
En este ejemplo usaremos un soporte mínimo de 0.2

In [5]:
from mlxtend.frequent_patterns import apriori

df_a = apriori(df, min_support = 0.2, use_colnames = True, verbose = 1)
df_a

Processing 42 combinations | Sampling itemset size 3


,support,itemsets
0,0.35,(BISCUIT)
1,0.20,(BOURNVITA)
2,0.65,(BREAD)
3,0.40,(COFFEE)
4,0.30,(CORNFLAKES)
5,0.25,(MAGGI)
6,0.25,(MILK)
7,0.30,(SUGER)
8,0.35,(TEA)
9,0.20,"(BISCUIT, BREAD)"


### Generación de itemsets fecuentes con el modelo FPGrowth

Una alternativa al modelo Apriori es el [FPGrowth](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/). Este modelo, al contrario que el Apriori, no genera candidatos a patrones frecuentes, si no que va extendiendo un arbol en profundidad para generar los itemsets, lo que lo hace mucho mas eficiente.

Al igual que con el modelo Apriori, debemos especificar un soporte mínimo para los intemsets generados. Si volvemos a usar un soporte mínimo de 0.2:

In [6]:
from mlxtend.frequent_patterns import fpgrowth

df_f = fpgrowth(df, min_support = 0.2, use_colnames = True, verbose = 1)
df_f

9 itemset(s) from tree conditioned on items ()
0 itemset(s) from tree conditioned on items (BREAD)
1 itemset(s) from tree conditioned on items (BISCUIT)
1 itemset(s) from tree conditioned on items (MILK)
1 itemset(s) from tree conditioned on items (CORNFLAKES)
1 itemset(s) from tree conditioned on items (TEA)
0 itemset(s) from tree conditioned on items (BOURNVITA)
1 itemset(s) from tree conditioned on items (MAGGI)
0 itemset(s) from tree conditioned on items (COFFEE)
2 itemset(s) from tree conditioned on items (SUGER)
0 itemset(s) from tree conditioned on items (SUGER, COFFEE)
0 itemset(s) from tree conditioned on items (SUGER, BREAD)


,support,itemsets
0,0.65,(BREAD)
1,0.35,(BISCUIT)
2,0.25,(MILK)
3,0.30,(CORNFLAKES)
4,0.35,(TEA)
5,0.20,(BOURNVITA)
6,0.25,(MAGGI)
7,0.40,(COFFEE)
8,0.30,(SUGER)
9,0.20,"(BISCUIT, BREAD)"


### Generación de reglas

Una vez hayados los itemsets frecuentes, podemos proceder a generar las reglas de asociación que relacionan dichos itemsets frecuentes. 

A la hora de generar las reglas, debemos definir un umbral mínimo para las reglas generadas. Este umbral puede definirse en terminos de diferentes métricas:

 - `support`: El soporte mide la frecuencia con la que la regla se aplica al conjunto de datos. Es decir, si la regla es $X \rightarrow Y$, el soporte se calcula como la proporción de transacciones que contienen tanto el antecedente $X$ como el consecuente $Y$.
    $$
        support(X \rightarrow Y) = \frac{\text{Numero de transacciones en las que aparecen }X\text{ e }Y}{\text{Numero de transacciones total}}
    $$

 - `confidence`: La confianza es una medida de la probabilidad condicional de que ocurra $Y$ dado que ha ocurrido $X$. Es decir, si la regla es $X \rightarrow Y$, la confianza será el porcentaje de veces que el producto $Y$ se compra, dado que se ha comprado el producto $X$.
    $$
        confidence(X \rightarrow Y) = \frac{support(X \cup Y)}{support(X)}
    $$

 - `lift`: El lift de una regla mide qué tan probable es que se compre $Y$ cuando se compra $X$, en comparación con la probabilidad de comprar $Y$ sin considerar la compra de $X$.
    $$
        lift(X \rightarrow Y) = \frac{confidence(X \rightarrow Y)}{support(Y)}
    $$

 - `leverage`: El leverage de una regla mide la diferencia entre la frecuencia observada de que el antecedente y el consecuente aparezcan juntos y la frecuencia esperada si fueran estadísticamente independientes.
    $$
        leverage(X \rightarrow Y) = support(X \rightarrow Y) - (support(X) \times support(Y))
    $$
 
 - `conviction`: La conviction de una regla de asociación es una métrica que intenta medir la fuerza de la implicación de la regla, considerando la frecuencia con la que el antecedente ocurre sin el consecuente. 
    $$
        conviction(X \rightarrow Y) = \frac{1 - support(Y)}{1 - confidence(X \rightarrow Y)}
    $$

 - `zhangs_metric`: La métria de Zhang es otra medida utilizada para evaluar el interés de las reglas de asociación teniendo en cuenta la coocurrencia y la coocurrencia de anetcendente y consecuente en el dataset. Fue propuesta por Zhang en el 2000 y busca abordar algunas limitaciones de otras métricas como la confianza y el lift, especialmente en situaciones con datos muy desequilibrados.
 
    $$
        zhangs(X \rightarrow Y) = \frac{leverage(X \rightarrow Y)}{max([support(X \rightarrow Y) \times (1 - support(X))], [support(X) \times (support(Y)-support(X \rightarrow Y))])}
    $$

En este ejemplo usaremos un nivel mínimo de confianza del 60%. 

In [7]:
from mlxtend.frequent_patterns import association_rules

df_rules = association_rules(df_a, metric = "confidence", min_threshold = 0.6)
df_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(MILK),(BREAD),0.25,0.65,0.2,0.800000,1.230769,1.0,0.0375,1.75,0.250000,0.285714,0.428571,0.553846
1,(SUGER),(BREAD),0.30,0.65,0.2,0.666667,1.025641,1.0,0.0050,1.05,0.035714,0.266667,0.047619,0.487179
2,(CORNFLAKES),(COFFEE),0.30,0.40,0.2,0.666667,1.666667,1.0,0.0800,1.80,0.571429,0.400000,0.444444,0.583333
3,(SUGER),(COFFEE),0.30,0.40,0.2,0.666667,1.666667,1.0,0.0800,1.80,0.571429,0.400000,0.444444,0.583333
4,(MAGGI),(TEA),0.25,0.35,0.2,0.800000,2.285714,1.0,0.1125,3.25,0.750000,0.500000,0.692308,0.685714


<div class="alert alert-info">

**Ejercicio:**

A partir de las reglas obtenidas en el paso anterior:

- ¿Cual es la probabilidad de ver ventas de azúcar? ¿Y de pan?
- ¿Cual es la probabilidad de que se compren juntos pan y azucar?
- ¿Que porcentaje de compradores de azucar compran tambien pan?

</div>

In [8]:
soporte_azucar = df_a[df_a['itemsets'] == frozenset({'SUGER'})]['support'].values[0]
print(f"Probabilidad de ver ventas de azúcar: {soporte_azucar:.2f}")

soporte_pan = df_a[df_a['itemsets'] == frozenset({'BREAD'})]['support'].values[0]
print(f"Probabilidad de ver ventas de pan: {soporte_pan:.2f}")

Probabilidad de ver ventas de azúcar: 0.30
Probabilidad de ver ventas de pan: 0.65


In [9]:
itemset_pan_azucar = frozenset({'BREAD', 'SUGER'})
soporte_conjunto = df_a[df_a['itemsets'] == itemset_pan_azucar]['support'].values[0]
print(f"Probabilidad de que se compren juntos pan y azúcar: {soporte_conjunto:.2f}")

Probabilidad de que se compren juntos pan y azúcar: 0.20


In [10]:
regla = df_rules[(df_rules['antecedents'] == frozenset({'SUGER'})) & 
                 (df_rules['consequents'] == frozenset({'BREAD'}))]

confianza = regla['confidence'].values[0]
print(f"Porcentaje de compradores de azúcar que también compran pan: {confianza:.2%}")

Porcentaje de compradores de azúcar que también compran pan: 66.67%


<div class="alert alert-info">

**Ejercicio:**

El fichero `pedidos.csv` contiene la información sobre 3.000.000 facturas de un supermecado. En el fichero `productos.csv` tenéis la correspondencia entre códigos de producto y el nombre de los mismos.

Si consideramos que un itemset es frecuente cuando aparece en al menos 300 transacciones:
- ¿Qué soporte debemos establecer para calcular los itemsets frecuentes? ¿Cuántos itemsets frecuentes encontramos? ¿Cuál de ellos tiene el soporte más alto?
- ¿Cuántos de los itemsets frecuentes contienen el producto “Seedless Red Grapes”? ¿Cuál de los que tienen más de dos items tiene el soporte más alto?
- ¿Cuántos de los itemset frecuentes son subconjuntos de otro itemset?

Determina el valor necesario de confianza para que se generen entre 150 y 200 reglas con el soporte anteriormente definido. Indica el número de transacciones en las que se verifica la regla con la confianza más alta y que, además, incluyan “Organic Strawberries” en el antecedente o el consecuente.
</div>

In [11]:
pedidos = pd.read_csv("./Datos/pedidos.csv")[:3000]
productos = pd.read_csv("./Datos/productos.csv", index_col='product_id')
pedidos["product_id"] = pedidos["product_id"].replace(productos.to_dict()["product_name"])
grouped = pedidos.groupby('order_id')['product_id'].apply(list)
encoder = TransactionEncoder()
datos_codificados = encoder.fit_transform(grouped)
df = pd.DataFrame(datos_codificados, columns=encoder.columns_)

df

,0% Greek Strained Yogurt,1 Apple + 1 Pear Fruit Bar,1% Lowfat Milk,100 Calorie Per Bag Popcorn,100% Apple Juice Original,100% Cranberry Juice,100% Guava Juice,100% Juice No Added Sugar Orange Tangerine,100% Juice No Sugar Added Apple,100% Lactose Free Fat Free Milk,...,"Yogurt, Nonfat, Organic, Plain",Yuba Tofu Skin,Yukon Gold Potatoes 5lb Bag,ZBar Organic Chocolate Brownie Energy Snack,Zero Calorie Cream Soda,Zero Calories Berry Nutrient Enhanced Water,Zinfandel,Zucchini Noodles,gel hand wash sea minerals,with Crispy Almonds Cereal
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
304,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
305,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
306,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [12]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
from itertools import combinations
from collections import defaultdict

# Parámetros
INPUT_PEDIDOS   = "./Datos/pedidos.csv"
INPUT_PRODUCTOS = "./Datos/productos.csv"
MIN_SUPPORT_TX  = 300            # mínimo absoluto de transacciones
CHUNKSIZE       = 500_000       # filas por bloque (ajusta según RAM)

# 1) Prepara mapeo product_id → nombre
productos = pd.read_csv(INPUT_PRODUCTOS, index_col="product_id")["product_name"].to_dict()

# 2) PASO 1: GENERAR CANDIDATOS FRECUENTES EN CADA BLOQUE
candidates = set()

reader = pd.read_csv(INPUT_PEDIDOS, chunksize=CHUNKSIZE)
for chunk in reader:
    # a) mapear nombres y agrupar por order_id
    chunk["product_name"] = chunk["product_id"].map(productos)
    grouped = chunk.groupby("order_id")["product_name"].apply(list)
    n_trx_chunk = len(grouped)
    # b) calcular min_support relativo para el bloque
    min_support_chunk = MIN_SUPPORT_TX / n_trx_chunk
    # c) transformar a matriz booleana
    from mlxtend.preprocessing import TransactionEncoder
    te = TransactionEncoder()
    X = te.fit_transform(grouped)
    df_bool = pd.DataFrame(X, columns=te.columns_)
    # d) extraer itemsets frecuentes con apriori
    freq_its = apriori(df_bool,
                       min_support=min_support_chunk,
                       use_colnames=True,
                       verbose=0)
    # e) añadir cada itemset como candidato
    for its in freq_its["itemsets"]:
        candidates.add(frozenset(its))

print(f"Candidatos únicos tras el pase 1: {len(candidates)} itemsets")

# 3) PASO 2: CONTAR CANDIDATOS EN TODO EL DATASET
#    -> recorremos otra vez en bloques y contamos cada candidato
counts = defaultdict(int)

reader = pd.read_csv(INPUT_PEDIDOS, chunksize=CHUNKSIZE)
for chunk in reader:
    chunk["product_name"] = chunk["product_id"].map(productos)
    grouped = chunk.groupby("order_id")["product_name"].apply(list)
    for trx in grouped:
        s_trx = set(trx)
        for cand in candidates:
            if cand.issubset(s_trx):
                counts[cand] += 1

# 4) FILTRAR LOS VERDADEROS ITEMSETS FRECUENTES
frequent_itemsets = [
    (its, cnt)
    for its, cnt in counts.items()
    if cnt >= MIN_SUPPORT_TX
]

n_trx_total = sum(1 for _ in pd.read_csv(INPUT_PEDIDOS, usecols=["order_id"]).groupby("order_id"))
min_support = MIN_SUPPORT_TX / n_trx_total

print(f"\nItemsets frecuentes finales: {len(frequent_itemsets)}")
max_its, max_cnt = max(frequent_itemsets, key=lambda x: x[1])
print(f"– El más frecuente: {set(max_its)} con {max_cnt} apariciones (soporte ≈ {max_cnt/n_trx_total:.4f})")

# 5) GENERAR REGLAS SOBRE ESTOS ITEMSETS
#    Convertimos a DataFrame para usar association_rules
df_its = pd.DataFrame({
    "itemsets": [frozenset(its) for its, _ in frequent_itemsets],
    "support":  [cnt / n_trx_total     for _, cnt in frequent_itemsets]
})

# Barrido de confianza para 150–200 reglas
conf_threshold = None
for thr in np.linspace(0.1, 1.0, 91):
    rules = association_rules(df_its, metric="confidence", min_threshold=thr)
    if 150 <= len(rules) <= 200:
        conf_threshold = thr
        break

rules = association_rules(df_its, metric="confidence", min_threshold=conf_threshold or 0.5)
print(f"\nUmbral de confianza ≈ {conf_threshold:.2f} → {len(rules)} reglas generadas")

# 6) Top regla que incluya “Organic Strawberries”
mask = rules["antecedents"].apply(lambda s: "Organic Strawberries" in s) | \
       rules["consequents"].apply(lambda s: "Organic Strawberries" in s)
straw_rules = rules[mask]
if not straw_rules.empty:
    best = straw_rules.loc[straw_rules["confidence"].idxmax()]
    apariciones = int(best["support"] * n_trx_total)
    print("Regla top con 'Organic Strawberries':")
    print(f"  {set(best['antecedents'])} → {set(best['consequents'])}")
    print(f"  soporte={best['support']:.4f}, confianza={best['confidence']:.2%}, transacciones≈{apariciones}")
else:
    print("No se encontró regla con 'Organic Strawberries'.")


KeyboardInterrupt: 